# Import Library 

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import chi2_contingency, mannwhitneyu

# Load dataset
df = pd.read_csv("merged_clinical_dataset.csv")

# sns.set(style="whitegrid")
# plt.rcParams["figure.figsize"] = (10, 5)

df['SurgeryType'] = df['SurgeryType'].fillna(0).astype(int)
df['ICU_Type'] = df['ICU_Type'].fillna(0).astype(int)

# ==== ICU_ADMISSION COLUMN ====
df['ICU_Admission'] = (
    (df['Outcome'] == 2) |
    (df['SurgeryType'] == 3) |
    (df['ICU_Type'].isin([1, 2, 3]))
).astype(int)


# 1. DATASET STRUCTURE 


In [ ]:
print("=== Dataset Shape ===")
print(f"Rows: {df.shape[0]}  |  Columns: {df.shape[1]}")
print("\n=== Variable Types ===")
print(df.dtypes.value_counts())
print("\n=== .info() Summary ===")
df.info()
print("\n=== Statistical Summary (Numerical Columns) ===")
display(df.describe().transpose())
print("\n=== Unique Value Counts (All Columns) ===")
display(df.nunique().sort_values())

# 2. ICU ADMISSION TARGET 


In [ ]:
icu_counts = df['ICU_Admission'].value_counts().sort_index()
icu_labels = ['Non-ICU', 'ICU']
icu_percent = (icu_counts / len(df)) * 100
print("\n=== ICU Admission Distribution (Custom Definition) ===")
for label, count, pct in zip(icu_labels, icu_counts, icu_percent):
    print(f"{label}: {count} patients ({pct:.1f}%)")

plt.figure(figsize=(6, 4))
sns.barplot(x=icu_labels, y=icu_counts.values, palette='Blues')
plt.title("ICU Admission Distribution")
plt.ylabel("Number of Patients")

# for i, v in enumerate(icu_counts.values):
#     plt.text(i, v + 10, f"{v}\n({v/df.shape[0]*100:.1f}%)", ha='center')

plt.tight_layout()
plt.show()

# 3. MISSING VALUE ANALYSIS 


In [ ]:
missing = df.isnull().sum()
missing_percent = (missing / len(df)) * 100
missing_df = pd.DataFrame({'Missing Count': missing, 'Missing %': missing_percent})
missing_df = missing_df[missing_df['Missing Count'] > 0].sort_values(by='Missing %', ascending=False)
print("\n=== Missing Values (Top Features) ===")
display(missing_df.head(20))
plt.figure(figsize=(14, 6))
sns.barplot(x=missing_df.index, y=missing_df['Missing %'], color='tomato')
plt.xticks(rotation=90)
plt.ylabel("Missing %")
plt.title("Missing Data Percentage by Feature")
# for i, v in enumerate(missing_df['Missing %']):
#     plt.text(i, v + 1, f"{v:.1f}%", ha='center', fontsize=8)
plt.tight_layout()
plt.show()

# 4. DEMOGRAPHICS: AGE, SEX, ASA, and surgery 


In [ ]:
# Create ICU label
df['ICU_Label'] = df['ICU_Admission'].map({0: "Non-ICU", 1: "ICU"})

# Split the data
df_non_icu = df[df['ICU_Label'] == "Non-ICU"]
df_icu = df[df['ICU_Label'] == "ICU"]

# Create side-by-side plots manually
fig, axes = plt.subplots(2, 1, figsize=(6, 6), sharey=True)

# Non-ICU plot
sns.kdeplot(data=df_non_icu, x="Age", fill=True, color="green", ax=axes[0])
axes[0].set_title("Age Distribution: Non-ICU")
axes[0].set_xlabel("Age")
axes[0].set_ylabel("Density")

# ICU plot
sns.kdeplot(data=df_icu, x="Age", fill=True, color="red", ax=axes[1])
axes[1].set_title("Age Distribution: ICU")
axes[1].set_xlabel("Age")
axes[1].set_ylabel("Density")
axes[1].yaxis.set_tick_params(labelleft=True)

plt.tight_layout()
plt.show()

df['SurgeryType'] = df['SurgeryType'].replace({
    0: 'No Surgery',
    1: 'Planned',
    2: 'Unplanned',
    3: 'Emergency',
})

# === Grouped Barplots for Categorical Variables (Side-by-Side) ===
cat_vars = ['Sex', 'ASA_Score', 'SurgeryType', 'ICU_Type']
for var in cat_vars:
    plt.figure(figsize=(6, 4))
    sns.countplot(data=df, x=var, hue="ICU_Admission", palette=["skyblue", "orange"])
    plt.title(f"{var} by ICU Admission")
    plt.xlabel(var)
    plt.ylabel("Patient Count")
    plt.legend(title="ICU Admission", labels=["Non-ICU", "ICU"])
    plt.tight_layout()
    plt.show()


# 5. STATISTICAL TESTS


In [ ]:
print("\n=== Mann-Whitney U Tests ===")
for col in ['Age', 'BMI']:
    group1 = df[df['ICU_Admission'] == 0][col].dropna()
    group2 = df[df['ICU_Admission'] == 1][col].dropna()
    stat, p = mannwhitneyu(group1, group2, alternative='two-sided')
    print(f"{col}: U={stat:.2f}, p-value={p:.4f}")

print("\n=== Chi-Square Tests ===")
for col in cat_vars:
    contingency = pd.crosstab(df[col], df['ICU_Admission'])
    chi2, p, dof, expected = chi2_contingency(contingency)
    print(f"{col}: chi²={chi2:.2f}, p-value={p:.4f}")

# 6. CORRELATION ANALYSIS


In [ ]:
correlations = df.corr(numeric_only=True)['ICU_Admission'].sort_values(key=abs, ascending=False)
lists_corr = [correlations[1:21], correlations[21:42], correlations[42:63], correlations[63:]]

# Plot each group
for i, top_corr in enumerate(lists_corr, 1):
    print(f"\n=== Top Correlated Features with ICU Admission (Group {i}) ===")
    display(top_corr)

    plt.figure(figsize=(10, 8))
    sns.barplot(x=top_corr.values, y=top_corr.index, palette='coolwarm')
    plt.title("Top Correlated Features with ICU Admission")
    plt.xlabel("Correlation")
    plt.tight_layout()
    plt.show()

# 7. ADVANCED VISUALIZATIONS 


In [ ]:
import textwrap

# ==== 7. DIAGNOSIS GROUP ANALYSIS ====
df['AgeGroup'] = pd.cut(df['Age'], bins=[0, 40, 60, 80, 120], labels=["<40", "40-60", "60-80", "80+"])
df['SurgeryLabel'] = df['SurgeryType'].map({1: "Planned", 2: "Unplanned", 3: "Emergency", 0: "No Surgery"})
df['SexLabel'] = df['Sex'].map({2: 'Male', 1: 'Female'})
diag_cols = [col for col in df.columns if col.startswith("diagnosis_group")]

# Clean Label formatting function
def clean_label(col):
    return '\n'.join(textwrap.wrap(col.replace('diagnosis_group_', '').replace('_', ' ').title(), 25))

def format_autopct(pct):
    total = sum(values)
    count = int(round(pct * total / 100.0))
    return f'{pct:.1f}%\n({count})'

# === Diagnosis × ICU × Sex (Horizontal Barplots) ===
plot_data_sex = df.melt(id_vars=['ICU_Admission', 'SexLabel'], value_vars=diag_cols, var_name='Diagnosis', value_name='Present')
plot_data_sex = plot_data_sex[plot_data_sex['Present'] == 1]
plot_data_sex['Diagnosis'] = plot_data_sex['Diagnosis'].apply(clean_label)

for sex in plot_data_sex['SexLabel'].dropna().unique():
    plt.figure(figsize=(10, 8))
    subset = plot_data_sex[plot_data_sex['SexLabel'] == sex]
    sns.countplot(data=subset, y='Diagnosis', hue='ICU_Admission', palette='Set1', order=subset['Diagnosis'].value_counts().index)
    plt.title(f"Diagnosis Group by ICU Admission × Sex: {sex}")
    plt.xlabel("Patient Count")
    plt.ylabel("Diagnosis Group")
    plt.legend(title="ICU Admission", labels=["Non-ICU", "ICU"], loc='upper left', bbox_to_anchor=(1, 1))
    plt.tight_layout()
    plt.show()

# === Diagnosis × ICU × AgeGroup (Horizontal Barplots) ===
plot_data_age = df.melt(id_vars=['ICU_Admission', 'AgeGroup'], value_vars=diag_cols, var_name='Diagnosis', value_name='Present')
plot_data_age = plot_data_age[plot_data_age['Present'] == 1]
plot_data_age['Diagnosis'] = plot_data_age['Diagnosis'].apply(clean_label)

for age in plot_data_age['AgeGroup'].dropna().unique():
    plt.figure(figsize=(10, 8))
    subset = plot_data_age[plot_data_age['AgeGroup'] == age]
    sns.countplot(data=subset, y='Diagnosis', hue='ICU_Admission', palette='coolwarm', order=subset['Diagnosis'].value_counts().index)
    plt.title(f"Diagnosis Group by ICU Admission × Age Group: {age}")
    plt.xlabel("Patient Count")
    plt.ylabel("Diagnosis Group")
    plt.legend(title="ICU Admission", labels=["Non-ICU", "ICU"], loc='upper left', bbox_to_anchor=(1, 1))
    plt.tight_layout()
    plt.show()

# === Surgery Type × ICU (stacked %) ===
surg_ct = pd.crosstab(df['SurgeryLabel'], df['ICU_Admission'], normalize='index') * 100
surg_ct.plot(kind='bar', stacked=True, color=['skyblue', 'orange'], figsize=(6, 4))
plt.title("ICU Admission by Surgery Type")
plt.ylabel("Percentage of Patients")
plt.xticks(rotation=0)
plt.legend(['Non-ICU', 'ICU'], loc='upper left', bbox_to_anchor=(1, 1))
plt.tight_layout()
plt.show()

# === Pie Charts per Diagnosis Group ===
pie_data = df[['ICU_Admission'] + diag_cols]
cols = 3
rows = (len(diag_cols) + 1) // cols

fig, axes = plt.subplots(rows, cols, figsize=(14, rows * 4))
axes = axes.flatten()

for i, diag in enumerate(diag_cols):
    counts = pie_data[pie_data[diag] == 1]['ICU_Admission'].value_counts().sort_index()
    values = [counts.get(0, 0), counts.get(1, 0)]
    labels = ['Non-ICU', 'ICU']
    axes[i].pie(values, labels=labels, autopct=format_autopct, startangle=90,
                colors=['skyblue', 'salmon'], textprops={'fontsize': 9})
    axes[i].set_title(clean_label(diag), fontsize=11)

# Remove unused axes
for j in range(i + 1, len(axes)):
    fig.delaxes(axes[j])

plt.suptitle("ICU vs Non-ICU Distribution per Diagnosis Group", fontsize=16)
plt.tight_layout(rect=[0.03, 0.03, 1, 0.97])
plt.show()


In [ ]:
df.to_csv('to_preperation_phase.csv', index=False)